## Week 1 assignment
**Dog breed classifier- Given a picture od a dog, classify which of 120 breeds it belongs to**

In [10]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting! just a test
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [11]:
# This file contains all the main external libs we'll use
from fastai.imports import *

In [12]:
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

In [13]:
PATH = "/content/clouderizer/fast.ai/data/competitions/dog-breed-identification"
sz=224

In [ ]:
os.listdir(PATH)

In [ ]:
!unzip /content/clouderizer/fast.ai/data/competitions/dog-breed-identification/train.zip -d /content/clouderizer/fast.ai/data/competitions/dog-breed-identification

In [17]:
labels=pd.read_csv('/content/clouderizer/fast.ai/data/competitions/dog-breed-identification/labels.csv')

In [ ]:
labels.head()

In [ ]:
labels.groupby('breed').count().sort_values('id',ascending=False)

In [21]:
labels_csv='/content/clouderizer/fast.ai/data/competitions/dog-breed-identification/labels.csv'
n=len(list(open(labels_csv)))-1
val_idxs=get_cv_idxs(n)

In [22]:
val_idxs
arch=resnext101_64
sz=224
bs=64

In [23]:
def get_data(sz,bs):
    tfms=tfms_from_model(arch,sz,max_zoom=1.1,aug_tfms=transforms_side_on)
    data=ImageClassifierData.from_csv(PATH,'train',labels_csv,val_idxs=val_idxs,tfms=tfms,suffix='.jpg',bs=bs)
    return data


    

In [24]:
data=get_data(sz,bs)

**Training with precompute=True**

In [25]:
learn=ConvLearner.pretrained(arch,data,precompute=True)

100%|██████████| 32/32 [01:10<00:00,  2.20s/it]


In [29]:
learn.fit(1e-2,3)

epoch      trn_loss   val_loss   accuracy   
    0      0.976684   0.389485   0.913894  
    1      0.442068   0.295509   0.923679  
    2      0.300674   0.270575   0.928082  



[array([0.27058]), 0.92808219096432]

**Setting precomute=False so that model can benifit from data augmentation**

In [30]:
learn.precompute=False

In [31]:
learn.fit(1e-2,3,cycle_len=1)

epoch      trn_loss   val_loss   accuracy   
    0      0.315018   0.239956   0.931018  
    1      0.290567   0.23554    0.928571  
    2      0.259107   0.232051   0.931507  



[array([0.23205]), 0.9315068484985665]

In [32]:
learn.save('224_pre')

**We then train the model with 299*299 pixels image size to avoid overfitting and further improve accuracy.**

In [33]:
learn.load('224_pre')
learn.set_data(get_data(299,bs))

In [38]:
learn.fit(1e-2,3,cycle_len=1)

epoch      trn_loss   val_loss   accuracy   
    0      0.258456   0.229565   0.934442  
    1      0.236093   0.224976   0.934442  
    2      0.217493   0.218956   0.937378  



[array([0.21896]), 0.9373776899858464]

**The accuracy is now about 93%, which is quite good considering there are 120 labels and not too much data to train on.**

## End